# Working with MRI data in Python
There is an entire suite of Python libraries for reading, processing, and analyzing MRI data, primarily centered around the nipy ecosystem. We have more information about these libraries in the Onboarding wiki.

In [1]:
import nibabel as nib

/Users/tsalo/anaconda/envs/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
